# L2M reports

- NBA produces the last two minute reports for games that are within 5 points in the last 2 minutes
- They have a structured table format for each game they are reporting on with some data about each play
- Our goal with this notebook is to pull that data from the website and structure it in CSVs

In [92]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [93]:
def get_soup(html_link):
    r = requests.get(html_link)
    soup = BeautifulSoup(r.content, "html.parser")
    return soup

In [94]:
def get_links(soup):
    links = []
    for a in soup.find_all('a', href=True):
        link = a['href']
        if 'L2MReport.html' in link:
            links.append(a)
    expanded_links = []
    for link in links:
        expanded_links.append((link['href'], link.get_text()))
    return expanded_links

In [115]:
def get_game_link_data(expanded_links):
    df = pd.DataFrame(expanded_links, columns = ['game_link', 'game_score'])

    #Had to fix 2 game_score's which had an extra comma in it when we scraped it. typical kinda messy data cleaning. 

    ix = df[df['game_link'] =='https://official.nba.com/l2m/L2MReport.html?gameId=0022201014'].index
    df.loc[ix, 'game_score'] = 'Nets 122, Nuggets 120'

    ix = df[df['game_link'] =='https://official.nba.com/l2m/L2MReport.html?gameId=0022200598'].index
    df.loc[ix, 'game_score'] = 'Pacers 116, Hornets 111'

    blazer_games = df[df['game_score'].apply(lambda x: 'trail' in x.lower())]['game_score']
    df.loc[blazer_games.index, 'game_score'] = blazer_games.apply(lambda x: x.replace('Trail Blazers', 'Trailblazers'))

    df['team_1_score'] = df['game_score'].apply(lambda x: x.split(',')[0].strip())
    df['team_2_score'] = df['game_score'].apply(lambda x: x.split(',')[1].strip())

    df['team_1_name' ] = df['team_1_score'].apply(lambda x: x.split(" ")[0].strip())
    df['team_1_score' ] = df['team_1_score'].apply(lambda x: x.split(" ")[1].strip())

    df['team_2_name' ] = df['team_2_score'].apply(lambda x: x.split(" ")[0].strip())
    df['team_2_score' ] = df['team_2_score'].apply(lambda x: x.split(" ")[1].strip())
    
    df['game_id'] = df['game_link'].apply(lambda x: x.split('?')[1].split('=')[1])
    df['game_id'] = df['game_id'].apply(lambda x: x.split('%')[0])
    
    return df

In [96]:
nba_22_23 = 'https://official.nba.com/2022-23-nba-officiating-last-two-minute-reports/'
nba_23_24 = 'https://official.nba.com/2023-24-nba-officiating-last-two-minute-reports/'

In [116]:
soup = get_soup(nba_22_23)
links = get_links(soup)
df = get_game_link_data(expanded_links)
df.to_csv('../data/nba_22_23_l2m_metadata.csv', index=False)

In [117]:
soup = get_soup(nba_23_24)
links = get_links(soup)
df = get_game_link_data(expanded_links)

In [118]:
df

,game_link,game_score,team_1_score,team_2_score,team_1_name,team_2_name,game_id
0,https://official.nba.com/l2m/L2MReport.html?ga...,"Nuggets 94, Heat 89",94,89,Nuggets,Heat,0042200405
1,https://official.nba.com/l2m/L2MReport.html?ga...,"Heat 111, Nuggets 108",111,108,Heat,Nuggets,0042200402
2,https://official.nba.com/l2m/L2MReport.html?ga...,"Celtics 104, Heat 103",104,103,Celtics,Heat,0042200306
3,https://official.nba.com/l2m/L2MReport.html?ga...,"Nuggets 113, Lakers 111",113,111,Nuggets,Lakers,0042200314
4,https://official.nba.com/l2m/L2MReport.html?ga...,"Heat 111, Celtics 105",111,105,Heat,Celtics,0042200302
...,...,...,...,...,...,...,...
469,https://official.nba.com/l2m/L2MReport.html?ga...,"Raptors 108, Cavaliers 105",108,105,Raptors,Cavaliers,0022200008
470,https://official.nba.com/l2m/L2MReport.html?ga...,"Grizzlies 115, Knicks 112 (OT)",115,112,Grizzlies,Knicks,0022200009
471,https://official.nba.com/l2m/L2MReport.html?ga...,"Timberwolves 115, Thunder 108",115,108,Timberwolves,Thunder,0022200010
472,https://official.nba.com/l2m/L2MReport.html?ga...,"Suns 107, Mavericks 105",107,105,Suns,Mavericks,0022200013
